<a href="https://colab.research.google.com/github/almogelias/DLHomework4/blob/main/DLHomework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial import distance

!git clone -s https://github.com/almogelias/DLHomework4.git DLHomework4



Cloning into 'DLHomework4'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


# Preprocess of diabetes.arff file

In [2]:

from scipy.io import arff
import pandas as pd
import os
from sklearn import preprocessing


#Load the data using "arff.loadarff" then convert it to dataframe

repository_path = os.path.join(os.getcwd(), 'DLHomework4')
train_diabetes_path = os.path.join(repository_path, 'diabetes.arff')

data = arff.loadarff(train_diabetes_path)
df = pd.DataFrame(data[0])

y_train = df['class'].replace({b'tested_positive':'1', b'tested_negative':'0'})

        
# Drop last column of a dataframe
x_train = df.iloc[: , :-1]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_scaled)
#x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=3)

In [3]:
#Define input dimensions
diabetes_dim = x_train.shape[1]


# Generator Model

In [4]:
#Given input of noise (latent) vector, the Generator produces an sample.
def build_generator():

    noise_shape = (8,) #1D array of size 100 (latent vector / noise)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    model = Sequential()

    model.add(Dense(diabetes_dim, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(diabetes_dim*diabetes_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(diabetes_dim*diabetes_dim*diabetes_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(diabetes_dim), activation='tanh'))
    #model.add(Reshape(diabetes_dim))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated sample
    
    return Model(noise, img)

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training

generator = build_generator()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 8)                 0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8)                 32        
_________________________________________________________________
dense_1 (Dense)              (None, 64)                576       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 512)               3

#Discriminator Model

In [5]:

#Given an input sample, the Discriminator outputs the likelihood of the sample being real.
#Binary classification - true or false (we're calling it validity)

def build_discriminator():

    diabetes_dim_shape = (diabetes_dim,)
    model1 = Sequential()

    model1.add(Dense(diabetes_dim, input_shape=diabetes_dim_shape))
    model1.add(Dense(diabetes_dim*diabetes_dim*diabetes_dim))
    model1.add(LeakyReLU(alpha=0.2))
    model1.add(Dense(diabetes_dim*diabetes_dim))
    model1.add(LeakyReLU(alpha=0.2))
    model1.add(Dense(1, activation='sigmoid'))
    model1.summary()

    diabetes_input = Input(shape=diabetes_dim)
    validity = model1(diabetes_input)

    return Model(diabetes_input, validity)
#The validity is the Discriminator’s guess of input being real or not.
discriminator = build_discriminator()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 512)               4608      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 37,577
Trainable params: 37,577
Non-trainable params: 0
__________________________________________________

#Training flow

In [6]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#samples and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    half_batch = int(batch_size / 2)
#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of samples from our true dataset, generating a set of samples from our
#Generator, feeding both set of samples into our Discriminator, and finally setting the
#loss parameters for both the real and fake samples, as well as the combined loss. 
    
    for epoch in range(epochs):

      # ---------------------
      #  Train Discriminator
      # ---------------------
      
      # Select a random half batch of real samples
      idx = np.random.randint(0, x_train.shape[0], half_batch)
      diabetes = x_train.iloc[idx]


      noise = np.random.normal(0, 1, (half_batch, 8))

      # Generate a half batch of fake samples
      gen_diabetes = generator.predict(noise)

      # Train the discriminator on real and fake samples, separately
      #Research showed that separate training is more effective. 
      d_loss_real = discriminator.train_on_batch(diabetes, np.ones((half_batch, 1)))
      d_loss_fake = discriminator.train_on_batch(gen_diabetes, np.zeros((half_batch, 1)))
    #take average loss from real and fake samples. 

      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
      euclidean_distance = distance.euclidean(d_loss_real,d_loss_fake)
      '''
      if (euclidean_distance > 1.1):
        print ("real: "+str(d_loss_real)+" , fake: "+str(d_loss_fake) + " , Duclidean distance: "+ str(euclidean_distance))
        print ()
        print("real diabetes values:")
        print(pd.DataFrame(diabetes))
        print("fake diabetes values:")
        print(pd.DataFrame(gen_diabetes))
      '''
      
      
     
        
      
    #And within the same loop we train our Generator, by setting the input noise and
    #ultimately training the Generator to have the Discriminator label its samples as valid
    #by specifying the gradient loss.
      # ---------------------
      #  Train Generator
      # ---------------------
    #Create noise vectors as input for generator. 
    #Create as many noise vectors as defined by the batch size. 
    #Based on normal distribution. Output will be of size (batch size, 100)
      noise = np.random.normal(0, 1, (batch_size, 8)) 

      # The generator wants the discriminator to label the generated samples
      # as valid (ones)
      #This is where the genrator is trying to trick discriminator into believing
      #the generated sample is true (hence value of 1 for y)
      valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

      # Generator is part of combined where it got directly linked with the discriminator
      # Train the generator with noise as x and 1 as y. 
      # Again, 1 as the output as it is adversarial and if generator did a great
      #job of folling the discriminator then the output would be 1 (true)
      g_loss = combined.train_on_batch(noise, valid_y)
      if (g_loss > 1):
        print("fake diabetes noise values that fooled the model:")
        print(pd.DataFrame(noise))

    #Additionally, in order for us to keep track of our training process, we print the
    #progress and save the sample sample output depending on the epoch interval specified.  
    #Plot the progress
      
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      


In [7]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) samples, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its samples.  
z = Input(shape=(8,))   #Our random input to the generator
test_if_diabete = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the samples generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(test_if_diabete)  #Validity check on the generated sample


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates samples => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=150, batch_size=32, save_interval=10)

#Save model for future use to generate fake samples
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample function.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_9 (Dense)              (None, 512)               4608      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                32832     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 37,577
Trainable params: 37,577
Non-trainable params: 0
__________________________________________________

# German Credit DB

In [8]:
'''

#Load the data using "arff.loadarff" then convert it to dataframe

repository_path = os.path.join(os.getcwd(), 'DLHomework4')
train_german_credit_path = os.path.join(repository_path, 'german_credit.arff')

data = arff.loadarff(train_german_credit_path)
df = pd.DataFrame(data[0])

y_train = df['21'].replace({b'1':'1', b'2':'2'})

        
# Drop last column of a dataframe
x_train = df.iloc[: , :-1]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_scaled)
#x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=3)
'''

'\n\n#Load the data using "arff.loadarff" then convert it to dataframe\n\nrepository_path = os.path.join(os.getcwd(), \'DLHomework4\')\ntrain_german_credit_path = os.path.join(repository_path, \'german_credit.arff\')\n\ndata = arff.loadarff(train_german_credit_path)\ndf = pd.DataFrame(data[0])\n\ny_train = df[\'21\'].replace({b\'1\':\'1\', b\'2\':\'2\'})\n\n        \n# Drop last column of a dataframe\nx_train = df.iloc[: , :-1]\nmin_max_scaler = preprocessing.MinMaxScaler()\nx_scaled = min_max_scaler.fit_transform(x_train)\nx_train = pd.DataFrame(x_scaled)\n#x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=3)\n'

# Random Forest model

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
Xtrain, Xtest, ytrain, ytest = train_test_split(x_train, y_train,
                                                 test_size=0.33, random_state=42)

def BB_Model(sample_Xtrain,sample_ytrain):
  model = RandomForestClassifier(n_estimators=100, random_state=0)
  
  model = RandomForestClassifier(n_estimators=1000)
  model.fit(Xtrain, ytrain)
  return model

model=BB_Model(Xtrain,ytrain)
ypred = model.predict(Xtest)
accuracy_score(ytest, ypred)

0.7637795275590551

# BUILD new generator

In [60]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras import activations

#Given input of noise (latent) vector, the Generator produces an sample.
def build_generator(noise_shape, desired_confidence_shape):

    input_noise = Input(shape=noise_shape)
    input_confidence = Input(shape=desired_confidence_shape)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    x = Dense(noise_shape[0])(input_noise)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.2)(x)
    x = Dense(noise_shape[0]*noise_shape[0])(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.2)(x)
    x = Dense(noise_shape[0]*noise_shape[0]*noise_shape[0])(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.2)(x)
    x = Dense(np.prod(noise_shape[0]))(x)
    x = activations.tanh(x)
    x = Model(inputs=input_noise, outputs=x)
    

    y = Dense(desired_confidence_shape[0])(input_confidence)
    y = LeakyReLU(alpha=0.2)(y)
    y = BatchNormalization(momentum=0.2)(y)
    y = Dense(desired_confidence_shape[0]*desired_confidence_shape[0])(y)
    y = LeakyReLU(alpha=0.2)(y)
    y = BatchNormalization(momentum=0.2)(y)
    y = Model(inputs=input_confidence, outputs=y)

    combined = concatenate([x.output, y.output])

    z = Dense(2, activation="relu")(combined)
    z = Dense(1, activation="linear")(z)
    outputs=Dense(8)(z)

    model = Model(inputs=[x.input, y.input], outputs=outputs)
    model.summary()
    
    return model

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training
noise_shape = (8,)
desired_confidence_shape=(1,)
generator = build_generator(noise_shape,desired_confidence_shape)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)


Model: "model_133"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_119 (InputLayer)          [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_318 (Dense)               (None, 8)            72          input_119[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_194 (LeakyReLU)     (None, 8)            0           dense_318[0][0]                  
__________________________________________________________________________________________________
batch_normalization_124 (BatchN (None, 8)            32          leaky_re_lu_194[0][0]            
__________________________________________________________________________________________

In [73]:

#Given an input sample, the Discriminator outputs the likelihood of the sample being real.
#Binary classification - true or false (we're calling it validity)

def build_discriminator(sample_shape, desired_confidence_shape):

    
    
    if (type(sample_shape)=='keras.engine.keras_tensor.KerasTensor'):

      x = Dense(sample_shape.shape)(sample_shape)
      x = Dense(sample_shape.shape*sample_shape.shape*sample_shape.shape)(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = Dense(sample_shape.shape*sample_shape.shape)(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = Model(inputs=input_sample, outputs=x)

      y = Dense(desired_confidence_shape.shape)(desired_confidence_shape)
      y = LeakyReLU(alpha=0.2)(y)
      y = Dense(desired_confidence_shape.shape*desired_confidence_shape.shape)(y)
      y = LeakyReLU(alpha=0.2)(y)
      y = Model(inputs=input_confidence, outputs=y)

      combined = concatenate([x.output, y.output])

      z = Dense(2, activation="sigmoid")(combined)

      model = Model(inputs=[x.input, y.input], outputs=z)
      model.summary()
      
      return model

    else:
      input_sample = Input(shape=sample_shape)
      input_confidence = Input(shape=desired_confidence_shape)
#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

      x = Dense(sample_shape[0])(input_sample)
      x = Dense(sample_shape[0]*sample_shape[0]*sample_shape[0])(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = Dense(sample_shape[0]*sample_shape[0])(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = Model(inputs=input_sample, outputs=x)

      y = Dense(desired_confidence_shape[0])(input_confidence)
      y = LeakyReLU(alpha=0.2)(y)
      y = Dense(desired_confidence_shape[0]*desired_confidence_shape[0])(y)
      y = LeakyReLU(alpha=0.2)(y)
      y = Model(inputs=input_confidence, outputs=y)

      combined = concatenate([x.output, y.output])

      z = Dense(2, activation="sigmoid")(combined)

      model = Model(inputs=[x.input, y.input], outputs=z)
      model.summary()
      
      return model

    #return Model(diabetes_input, validity)
#The validity is the Discriminator’s guess of input being real or not.
noise_shape = (8,)
desired_confidence_shape=(1,)
discriminator = build_discriminator(noise_shape,desired_confidence_shape)

Model: "model_145"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_129 (InputLayer)          [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_348 (Dense)               (None, 8)            72          input_129[0][0]                  
__________________________________________________________________________________________________
input_130 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_349 (Dense)               (None, 512)          4608        dense_348[0][0]                  
__________________________________________________________________________________________

In [30]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#samples and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    half_batch = int(batch_size / 2)
#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of samples from our true dataset, generating a set of samples from our
#Generator, feeding both set of samples into our Discriminator, and finally setting the
#loss parameters for both the real and fake samples, as well as the combined loss. 
    
    for epoch in range(epochs):

      # ---------------------
      #  Train Discriminator
      # ---------------------
      '''
      # Select a random half batch of real samples

      idx = np.random.randint(0, Xtrain.shape[0], half_batch)
      diabetes = Xtrain.iloc[idx]

      '''
   

      noise = np.random.normal(0, 1, (half_batch, 8))
      scalar = np.random.normal(0, 1, (half_batch, 8))
      # Generate a half batch of fake samples
      gen_diabetes = generator.predict(noise,scalar)

      # Train the discriminator on real and fake samples, separately
      #Research showed that separate training is more effective. 
      d_loss_one = discriminator.train_on_batch(gen_diabetes, np.ones((half_batch, 1)))
      d_loss_zero = discriminator.train_on_batch(gen_diabetes, np.zeros((half_batch, 1)))
    #take average loss from real and fake samples. 

      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
      euclidean_distance = distance.euclidean(d_loss_real,d_loss_fake)
      '''
      if (euclidean_distance > 1.1):
        print ("real: "+str(d_loss_real)+" , fake: "+str(d_loss_fake) + " , Duclidean distance: "+ str(euclidean_distance))
        print ()
        print("real diabetes values:")
        print(pd.DataFrame(diabetes))
        print("fake diabetes values:")
        print(pd.DataFrame(gen_diabetes))
      '''
      
      
     
        
      
    #And within the same loop we train our Generator, by setting the input noise and
    #ultimately training the Generator to have the Discriminator label its samples as valid
    #by specifying the gradient loss.
      # ---------------------
      #  Train Generator
      # ---------------------
    #Create noise vectors as input for generator. 
    #Create as many noise vectors as defined by the batch size. 
    #Based on normal distribution. Output will be of size (batch size, 100)
      noise = np.random.normal(0, 1, (batch_size, 8)) 

      # The generator wants the discriminator to label the generated samples
      # as valid (ones)
      #This is where the genrator is trying to trick discriminator into believing
      #the generated sample is true (hence value of 1 for y)
      valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

      # Generator is part of combined where it got directly linked with the discriminator
      # Train the generator with noise as x and 1 as y. 
      # Again, 1 as the output as it is adversarial and if generator did a great
      #job of folling the discriminator then the output would be 1 (true)
      g_loss = combined.train_on_batch(noise, valid_y)
      if (g_loss > 1):
        print("fake diabetes noise values that fooled the model:")
        print(pd.DataFrame(noise))

    #Additionally, in order for us to keep track of our training process, we print the
    #progress and save the sample sample output depending on the epoch interval specified.  
    #Plot the progress
      
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      


In [74]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
noise_shape = (8,)
desired_confidence_shape=(1,)
discriminator = build_discriminator(noise_shape,desired_confidence_shape)
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) samples, let us not track any metrics.
generator = build_generator(noise_shape,desired_confidence_shape)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its samples.  
z = Input(shape=(8,)) 
c = Input(shape=(1,))  #Our random input to the generator
test_if_diabete = generator([z,c])

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the samples generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(test_if_diabete,c)  #Validity check on the generated sample


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates samples => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=150, batch_size=32, save_interval=10)

#Save model for future use to generate fake samples
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model2.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample function.

Model: "model_148"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_131 (InputLayer)          [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_354 (Dense)               (None, 8)            72          input_131[0][0]                  
__________________________________________________________________________________________________
input_132 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_355 (Dense)               (None, 512)          4608        dense_354[0][0]                  
__________________________________________________________________________________________

AssertionError: ignored